In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
# from tensorflow.keras.applications import MobileNetV2
# from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
directory = ('/kaggle/input/sign-language-gesture-images-dataset/Gesture Image Data/')
directory

In [3]:
dict_labels = {
    '_':1,
    '0':2,
    '1':3,
    '2':4,
    '3':5,
    '4':6,
    '5':7,
    '6':8,
    '7':9,
    '8':10,
    '9':11,
    'A':12,
    'B':13,
    'C':14,
    'D':15,
    'E':16,
    'F':17,
    'G':18,
    'H':19,
    'I':20,
    'J':21,
    'K':22,
    'L':23,
    'M':24,
    'N':25,
    'O':26,
    'P':27,
    'Q':28,
    'R':29,
    'S':30,
    'T':31,
    'U':32,
    'V':33,
    'W':34,
    'X':35,
    'Y':36,
    'Z':37   
}

In [ ]:
# images, labels = [], []
# for ix in os.listdir(directory):
#     image_path = os.path.join(directory, ix)
#     for cx in os.listdir(image_path):
#         img_path = os.path.join(image_path, cx)
#         img = cv2.imread(img_path, 0)
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         img = img.reshape((50,50,3))
#         img = img/255.0
#         images.append(img)
#         labels.append(dict_labels[ix])

In [4]:
images = []
labels = []
for label in os.listdir(directory):
    label_path = os.path.join(directory, label)
    for img in os.listdir(label_path):
        image_path = os.path.join(label_path, img)
        img = load_img(image_path, target_size=(50, 50))
        img = img_to_array(img)
        images.append(img)
        labels.append(dict_labels[label])
#     print(label, "complete")

In [5]:
images = np.array(images)
labels = np.array(labels)

In [6]:
print("Shape of data: {}".format(images.shape))

In [7]:
labels = to_categorical(labels, num_classes = 38)

In [8]:
images, labels = shuffle(images, labels)

In [10]:
images = images / 255.0

In [11]:
# split the data into train and test data
# train-size: 80%
# test/val-size: 20%
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2)

In [12]:
print("Training images shape: {} || Training labels shape: {}".format(x_train.shape, y_train.shape))
print("Test images shape: {} || Test labels shape: {}".format(x_test.shape, y_test.shape))

In [14]:
model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=(50,50,3)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(38, activation='softmax'))

In [15]:
model.summary()

In [16]:
# compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [17]:
# set callbacks
callback = EarlyStopping(
    monitor='loss',
    patience=3
)

In [18]:
# fit the train data into the model
history = model.fit(
    x_train, y_train,
    epochs=20,
    callbacks=[callback],
    batch_size=32,
    validation_data=(x_test, y_test)
)

In [19]:
len(history.history['loss'])

In [20]:
y_pred_custom = model.predict(x_test)
acc = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred_custom, axis=1))
acc*100

In [22]:
# save the model
model.save('alphanumeralNet.h5')

In [21]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Accuracy/Loss")
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['train_accuracy','test_accuracy','loss', 'validation_loss'])
plt.show()